In [1]:
import pandas as pd
from numpy import *
import time
from texttable import Texttable
def readFile(filename):
    files = open(filename, "r", encoding="ISO-8859-1")
    data = []
    for line in files.readlines():
        item = line.strip().split("::")
        data.append(item)
    return data
movies = readFile("/Users/chai/Desktop/cf/movies.dat")

In [2]:
print(movies[1])

['2', 'Jumanji (1995)', "Adventure|Children's|Fantasy"]


In [3]:
class CF:
    def __init__(self, movies, ratings, k=5, n=10):
        self.movies = movies
        self.ratings = ratings
        self.k = k
        self.n = n
        self.userDict = {}
        self.ItemUser = {}
        self.neighbors = []
        self.recommandList = []
        self.cost = 0.0
    def recommendByUser(self, userId):
        self.formatRate()
        self.n = len(self.userDict[userId])
        self.getNearestNeighbor(userId)
        self.getrecommandList(userId)
        self.getPrecision(userId)

    def getrecommandList(self, userId):
        self.recommandList = []
        recommandDict = {}
        for neighbor in self.neighbors:
            movies = self.userDict[neighbor[1]]
            for movie in movies:
                if(movie[0] in recommandDict):
                    recommandDict[movie[0]] += neighbor[0]
                else:
                    recommandDict[movie[0]] = neighbor[0]
        for key in recommandDict:
            self.recommandList.append([recommandDict[key], key])
        self.recommandList.sort(reverse=True)
        self.recommandList = self.recommandList[:self.n]

    def formatRate(self):
        self.userDict = {}
        self.ItemUser = {}
        for i in self.ratings:
            temp = (i[1], float(i[2]) / 5)
            if(i[0] in self.userDict):
                self.userDict[i[0]].append(temp)
            else:
                self.userDict[i[0]] = [temp]
            if(i[1] in self.ItemUser):
                self.ItemUser[i[1]].append(i[0])
            else:
                self.ItemUser[i[1]] = [i[0]]

    def getNearestNeighbor(self, userId):
        neighbors = []
        self.neighbors = []
        for i in self.userDict[userId]:
            for j in self.ItemUser[i[0]]:
                if(j != userId and j not in neighbors):
                    neighbors.append(j)
        for i in neighbors:
            dist = self.getCost(userId, i)
            self.neighbors.append([dist, i])
        self.neighbors.sort(reverse=True)
        self.neighbors = self.neighbors[:self.k]

    def formatuserDict(self, userId, l):
        user = {}
        for i in self.userDict[userId]:
            user[i[0]] = [i[1], 0]
        for j in self.userDict[l]:
            if(j[0] not in user):
                user[j[0]] = [0, j[1]]
            else:
                user[j[0]][1] = j[1]
        return user

    def getCost(self, userId, l):
        user = self.formatuserDict(userId, l)
        x = 0.0
        y = 0.0
        z = 0.0
        for k, v in user.items():
            x += float(v[0]) * float(v[0])
            y += float(v[1]) * float(v[1])
            z += float(v[0]) * float(v[1])
        if(z == 0.0):
            return 0
        return z / sqrt(x * y)

    def getPrecision(self, userId):
        user = [i[0] for i in self.userDict[userId]]
        recommand = [i[1] for i in self.recommandList]
        count = 0.0
        if(len(user) >= len(recommand)):
            for i in recommand:
                if(i in user):
                    count += 1.0
            self.cost = count / len(recommand)
        else:
            for i in user:
                if(i in recommand):
                    count += 1.0
            self.cost = count / len(user)
    def showTable(self):
        neighbors_id = [i[1] for i in self.neighbors]
        rows = []
        fromID = []
        for item in self.recommandList:
            fromID = []
            for i in self.movies:
                if i[0] == item[1]:
                    movie = i
                    break
            for i in self.ItemUser[item[1]]:
                if i in neighbors_id:
                    fromID.append(i)
            movie.append(fromID)
            movie.append(item[0])
            rows.append(movie)
        table = pd.DataFrame(data=rows, columns=('MovieId', 'Name', 'genre', 'related people','similarity'))
        table.to_csv("/Users/chai/Desktop/Data.csv")
    def showRecommand(self):
        movies = []
        for item in self.recommandList:
            for i in self.movies:
                if i[0] == item[1]:
                    movie = i
                    break
            rows.append(movie)
        return movies

In [4]:
def averagenum(num):
    nsum = 0
    for i in range(len(num)):
        nsum += num[i]
    return nsum / len(num)
start = time.clock()
movies = readFile("/Users/chai/Desktop/cf/movies.dat")
ratings = readFile("/Users/chai/Desktop/cf/ratings.dat")
demo = CF(movies, ratings, k=20)
User = "6040"
demo.formatRate()
whole = demo.userDict
whole_data = [i for i in range(1,6041)]
ratings[-1]
from sklearn.model_selection import KFold
New_sam = KFold(n_splits=5, shuffle=True)
for index1, index2 in New_sam.split(whole_data):
    train_set = []
    test_set = []
    for i in range(len(whole_data)):
        if(i in index1):
            train_set.append(whole_data[i])
        if(i in index2):
            test_set.append(whole_data[i])
    break
train = {}
test = {}
for i in train_set:
    s = str(i)
    train[s] = whole[s]
for i in test_set:
    s = str(i)
    test[s] = whole[s]
demo.userDict = train
accuracy = []
count = 0
for i in test_set:
    if count <10:
        USER = str(i)
        demo.recommendByUser(USER)
        accuracy.append(demo.cost)
        count = count+1
    else:
        break
acc = averagenum(accuracy)
print(acc)


0.5245393516143764


In [5]:
data = pd.read_csv("/Users/chai/Desktop/Data.csv")

In [6]:
data

,Unnamed: 0,MovieId,Name,genre,related people,similarity
0,0,457,"Fugitive, The (1993)",Action|Thriller,"['48', '301', '338', '403', '676', '777', '791...",7.754383
1,1,356,Forrest Gump (1994),Comedy|Romance|War,"['48', '301', '338', '403', '676', '777', '791...",7.754383
2,2,3255,"League of Their Own, A (1992)",Comedy|Drama,"['48', '301', '338', '403', '676', '777', '791...",7.754383
3,3,1610,"Hunt for Red October, The (1990)",Action|Thriller,"['48', '301', '338', '403', '676', '777', '791...",7.754383
4,4,1580,Men in Black (1997),Action|Adventure|Comedy|Sci-Fi,"['48', '301', '338', '403', '676', '777', '791...",7.754383
5,5,1270,Back to the Future (1985),Comedy|Sci-Fi,"['48', '301', '338', '403', '676', '777', '791...",7.754383
6,6,1,Toy Story (1995),Animation|Children's|Comedy,"['48', '301', '338', '403', '676', '777', '791...",7.754383
7,7,480,Jurassic Park (1993),Action|Adventure|Sci-Fi,"['48', '301', '338', '403', '676', '777', '791...",7.381439
8,8,1517,Austin Powers: International Man of Mystery (1...,Comedy,"['48', '301', '338', '403', '676', '777', '791...",7.381439
9,9,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War,"['48', '301', '338', '403', '676', '777', '791...",7.381439
